In [1]:
import pandas as pd

# Extract /textit{} contents from .tex

In [2]:
import csv, re

In [3]:
import os

In [ ]:
def extract_textit_sentence(tex_file, csv_file):
    with open(tex_file, 'r',  errors='replace') as f:
        tex = f.read()
    if tex:
        doc = tex.split("begin{document}")
        if (len(doc) >= 2) :
            matches = re.findall(r'([^.]*?\\textit\{([^}]*)\}[^.]*\.)', doc[1])
            with open(csv_file, 'a') as csvfile:
                writer = csv.writer(csvfile, delimiter=',')
                for match in matches:
                    sentence, textit_content = match
                    writer.writerow([textit_content, sentence, tex_file])

In [ ]:
# try with a single local tex file
with open("output_1501.00565.csv", 'w') as csvfile:
        writer = csv.writer(csvfile, delimiter=',')
        writer.writerow(["textit_content", "sentence", "file_name"])
extract_textit_sentence('sampled_data/que_tex/1501.00565.tex', 'output_1501.00565.csv')

In [ ]:
with open("textit_sent_file.csv", 'w') as csvfile:
        writer = csv.writer(csvfile, delimiter=',')
        writer.writerow(["textit_content", "sentence", "file_name"])
tex_dir = '/home/shufan/Projects/def_arxiv/sampled_data/que_tex'
for subdir, dirs, filenames in os.walk(tex_dir):
    for filename in filenames:
        filepath = "sampled_data/que_tex/" + filename
        try:
            extract_textit_sentence(filepath, 'textit_sent_file.csv')
        except (UnicodeDecodeError) as error:
            print(filepath + error.message)
            continue
        except (FileNotFoundError) as error:
            print(filepath + " NOT FOUND!!")
            continue
            

In [ ]:
def extract_definitions(tex_file, csv_file):
    with open(tex_file, 'r', errors='replace') as f:
        tex_content = f.read()
    doc = tex_content.split("begin{document}")
    if (len(doc) >= 2) :
        doc = doc[1]
        definitions = re.findall(r'\\begin{definition}\[(.*?)](.*?)\\end{definition}', doc, re.DOTALL)
        
        theorem_sentences = re.findall(r'\\begin{theorem}(.*?)\\end{theorem}', doc, re.DOTALL)
        thm_it_sentences = [sent for sent in theorem_sentences if '\\textit{' in sent]
        thm_it_terms = [re.findall(r'\\textit{(.*?)}', sent) for sent in thm_it_sentences]
        thm_emph_sentences = [sent for sent in theorem_sentences if '\\emph{' in sent]
        thm_emph_terms = [re.findall(r'\\emph{(.*?)}', sent) for sent in thm_emph_sentences]
        
        def_sentences = re.findall(r'\\begin{definition}(.*?)\\end{definition}', doc, re.DOTALL)
        def_it_sentences = [sent for sent in def_sentences if '\\textit{' in sent]
        def_it_terms = [re.findall(r'\\textit{(.*?)}', sent) for sent in def_it_sentences]
        def_emph_sentences = [sent for sent in def_sentences if '\\emph{' in sent]
        def_emph_terms = [re.findall(r'\\emph{(.*?)}', sent) for sent in def_emph_sentences]
        with open(csv_file, 'a') as csvfile:
            writer = csv.writer(csvfile, delimiter=',')
            for definition in definitions:
                writer.writerow([definition[0].strip(), definition[1].strip(), tex_file, "definition[]"])
            for sent, term_list in zip(thm_it_sentences, thm_it_terms):
                writer.writerow([';'.join(term_list), sent, tex_file, "theorem + \\textit{}"])
            for sent, term_list in zip(thm_emph_sentences, thm_emph_terms):
                writer.writerow([';'.join(term_list), sent, tex_file, "theorem + \\emph{}"])
            for sent, term_list in zip(def_it_sentences, def_it_terms):
                writer.writerow([';'.join(term_list), sent, tex_file, "definition + \\textit{}"])
            for sent, term_list in zip(def_emph_sentences, def_emph_terms):
                writer.writerow([';'.join(term_list), sent, tex_file, "definition + \\emph{}"])

In [ ]:
with open("output_def.csv", 'w') as csvfile:
        writer = csv.writer(csvfile, delimiter=',')
        writer.writerow(["term", "def", "file_name", "extraction_rule"])
extract_definitions('sampled_data/que_tex/paper_fullerenes-ipr.tex', 'output_def.csv')


In [ ]:
with open("term_def_file.csv", 'w') as csvfile:
        writer = csv.writer(csvfile, delimiter=',')
        writer.writerow(["term", "def", "file_name", "extraction_rule"])
tex_dir = '/home/shufan/Projects/def_arxiv/sampled_data/que_tex'
for subdir, dirs, filenames in os.walk(tex_dir):
    for filename in filenames:
        filepath = "sampled_data/que_tex/" + filename
        try:
            extract_definitions(filepath, 'term_def_file.csv')
        except (UnicodeDecodeError) as error:
            print(filepath + error.message)
            continue
        except (FileNotFoundError) as error:
            print(filepath + " NOT FOUND!!")
            continue

# Examine extracted term-def pairs

In [ ]:
df_all_def = pd.read_csv('out_def_all.csv', lineterminator='\n', delimiter=',')
df_all_def.describe()

In [ ]:
df_all_def['extraction_rule\r'].value_counts()

In [ ]:
df_all_def['term'].value_counts()[:60]

In [ ]:
df_clean = df_all_def[~df_all_def['term'].isin(["(i);(ii)", "i.e.", '\\', "not", "good", "(;)", "(i);(ii);(iii)", "\\textbf{Pr;\\textbf{Pr", "all", "type", "K\\\"{o", "et al.", "\\cite{AS17}" ])]

In [ ]:
df_clean = df_clean[ False  == df_clean["term"].str.contains("cite{") ]

In [ ]:
df_clean = df_clean[~df_clean['term'].isin(["mod", "any", "\\", "\d;\d" ])]

In [ ]:
df_clean['term'].value_counts()[:60]

In [ ]:
df_clean[df_clean['term'].str.len() == 1]

In [ ]:
df_clean = df_clean[df_clean['term'].str.len() > 1]

In [ ]:
df_clean['extraction_rule\r'].value_counts()[:60]

In [ ]:
df_clean.to_csv("cleaned_term_def.csv", index=False)

# Remove latex tags => convert latex into plain text

In [ ]:
#! pip install pylatexenc

In [4]:
from pylatexenc.latex2text import LatexNodes2Text

In [5]:
df_clean = pd.read_csv('cleaned_term_def.csv', lineterminator='\n', delimiter=',')


In [6]:
df_clean.dropna(
    axis=0,
    how='any', #drop all rows with just a empty cell(s)
    inplace=True
)

df_clean

,term,def,file_name,extraction_rule\r
0,admissible $G$-cover,\label{def_admissibleGcover}\nLet $G$ be a fin...,/home/shufan/que_tex/abelian_tropical_covers.tex,definition + \emph{}\r
1,"$\mathfrak{C} (S, k), \text{ k-cut multi-set o...","%$$\mathfrak{C} (S, k) = \{\pi(S): S \subseteq...",/home/shufan/que_tex/CSPpaper.tex,definition[]\r
2,$\leq \text{on graphs}$,\begin{itemize}\n$$$$\n\item Assign Graph G th...,/home/shufan/que_tex/CSPpaper.tex,definition[]\r
3,light reduction pair;heavy reduction pair,"\n Let $\pi\in\fs_n$ and let $x,y$ be a pair ...",/home/shufan/que_tex/Post071005.tex,definition + \emph{}\r
4,Testability for graph properties,\label{testgraph}\cite{RS} A graph property $...,/home/shufan/que_tex/hereditary_submit.tex,definition[]\r
...,...,...,...,...
17851,Irreducibility,\label{d:irred}\nA matrix $A\in\Rpnn$ is calle...,/home/shufan/que_tex/1307.3716.tex,definition[]\r
17852,Transient of matrices,\label{d:transient}\nThe least nonnegative~$t$...,/home/shufan/que_tex/1307.3716.tex,definition[]\r
17853,Row and column transients,\label{d:rowcolumn}\nLet $A\in\Rpnn$.\nThe lea...,/home/shufan/que_tex/1307.3716.tex,definition[]\r
17854,Factor rank in max algebra,"In max algebra, {\rm factor rank\/} of $A\in\R...",/home/shufan/que_tex/1307.3716.tex,definition[]\r


In [7]:
def insert_space_before_and_after_emph_or_textit(s):
    # insert_space_after_brace
    after_pattern = r'(\\(?:emph|textit)\{[^}]*\})([.,;)\]]|$)'
    
    # Use the re.sub function to insert a space after } for all matching substrings
    result = re.sub(after_pattern, r'\1 \2', s)
    
    # insert_space_after_parenthesis_or_bracket(s) :
    before_pattern = r'(\(|\[)(\\(?:emph|textit)\{)'
    
    # Use the re.sub function to insert a space after ( or [ for all matching substrings
    result = re.sub(before_pattern, r'\1 \2', result)
    
    return result

In [8]:
df_clean['def'] = df_clean['def'].apply(insert_space_before_and_after_emph_or_textit)


In [9]:
df_clean['plain_text_def'] = df_clean['def'].apply(LatexNodes2Text().latex_to_text)

In [10]:
df_clean['plain_text_term'] = df_clean['term'].apply(LatexNodes2Text().latex_to_text)

In [11]:
df_clean['plain_text_def'].value_counts()[:20]

\n\n                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            106
\n                                                                                                                                                                                                                                                                                                                                                                                                                                                                  

In [12]:
# Remove rows where 'def' contains only '\n' or repeated '\n'
df_clean = df_clean[~df_clean['plain_text_def'].str.match(r'^\n+$')]
df_clean = df_clean[~df_clean['plain_text_def'].str.match(r'^ \n+$')]

In [ ]:
df_clean['plain_text_def'].value_counts()[:20]

# Annotate definitions

In [14]:
df_clean.rename(columns={"extraction_rule\r": "extraction_rule"}, inplace=True)

In [15]:
df_clean['extraction_rule'] = df_clean['extraction_rule'].str.rstrip('\r')

In [16]:
def mask_emph(text):
    pattern = r'\\emph\{(.*?)\}'

    def replace(match):
        words = match.group(1).split()
        if not words:
            return ''
        masked_words = ['B_MATH_TERM'] + ['I_MATH_TERM'] * (len(words) - 1)
        return ' '.join(masked_words)
    return re.sub(pattern, replace, text)

# Example usage
text = r'This is some text with \emph{emphasized words A B C D} in it. \emph{blah}. \emph{blah blah}'
masked_text = mask_emph(text)
print(masked_text)

This is some text with B_MATH_TERM I_MATH_TERM I_MATH_TERM I_MATH_TERM I_MATH_TERM I_MATH_TERM in it. B_MATH_TERM. B_MATH_TERM I_MATH_TERM


In [17]:
def mask_textit(text):
    pattern = r'\\textit\{(.*?)\}'

    def replace(match):
        words = match.group(1).split()
        if not words:
            return ''
        masked_words = ['B_MATH_TERM'] + ['I_MATH_TERM'] * (len(words) - 1)
        return ' '.join(masked_words)
    return re.sub(pattern, replace, text)

# Example usage
text = r'\textit{blah blah}! This is some text with \textit{emphasized words A B C D} in it. \textit{blah}. \textit{blah blah}'
masked_text = mask_textit(text)
print(masked_text)

B_MATH_TERM I_MATH_TERM! This is some text with B_MATH_TERM I_MATH_TERM I_MATH_TERM I_MATH_TERM I_MATH_TERM I_MATH_TERM in it. B_MATH_TERM. B_MATH_TERM I_MATH_TERM


In [18]:
def mask_substring(text, substring):
    pattern = re.compile(re.escape(substring), re.IGNORECASE)
    def replace(match):
        words = match.group().split()
        if not words:
            return ''
        masked_words = ['B_MATH_TERM'] + ['I_MATH_TERM'] * (len(words) - 1)
        return ' '.join(masked_words)
    return pattern.sub(replace, text)

# Example usage
text = 'This is some text with a SubString A in it. SubString A comes again.'
substring = 'substring a'
masked_text = mask_substring(text, substring)
print(masked_text)

This is some text with a B_MATH_TERM I_MATH_TERM in it. B_MATH_TERM I_MATH_TERM comes again.


In [19]:
def label_text(text):
    words = text.split()
    masked_words = ['O' if word not in ('B_MATH_TERM', 'I_MATH_TERM') else word for word in words]
    return ' '.join(masked_words)

'''
def label_text(text):
    B_MASK = "B_MATH_TERM"
    I_MASK = "I_MATH_TERM"
    text = text.replace(B_MASK, f" {B_MASK} ")
    text = text.replace(I_MASK, f" {I_MASK} ")
    words = text.split()
    masked_words = ['O' if word not in ('B_MATH_TERM', 'I_MATH_TERM') else word for word in words]
    return ' '.join(masked_words)

'''

'''def label_text(text):
    words = text.split()
    masked_words = []
    for word in words:
        if word.contains('B_MATH_TERM'):
            masked_words.append('B_MATH_TERM')
        elif word.contains('I_MATH_TERM'):
            masked_words.append('B_MATH_TERM')
        else:
            masked_words.append('O')
    return ' '.join(masked_words)'''

# Example usage
text = 'This is some text with B_MATH_TERM , and I_MATH_TERM in it.'
labeled_text = label_text(text)
print(labeled_text)

O O O O O B_MATH_TERM O O I_MATH_TERM O O


In [20]:
#Normally this should be corrected by previous space adding.
text2 = 'If G has the property that, for any two edges {a, b},{c, d}, there is an automorphism α such that α(a) = c and α(b) = d and also an automorphism β such that β(a) = d and β(b) = c, then G is said to be B_MATH_TERM.'
label_text(text2)

'O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O'

In [21]:
def mask_def_sentence(sent, extraction_rule, term=''):
    if 'emph' in extraction_rule:
        masked_text = mask_emph(sent)
    elif 'textit' in extraction_rule:
        masked_text = mask_textit(sent)
    else:
        masked_text = mask_substring(sent, term)
    return LatexNodes2Text().latex_to_text(masked_text) # convert to plain text

In [22]:
df_clean['masked_plain_text_def'] = df_clean.apply(lambda x: mask_def_sentence(x['def'], x['extraction_rule'], x['term']), axis=1)

In [ ]:
df_clean['masked_plain_text_def'].loc[:10]

In [23]:
df_clean[~df_clean['masked_plain_text_def'].str.contains('B_MATH_TERM')].to_csv("cleaned_term_def_but_unlabeled.csv", index=False)
# some definitions might be commented TODO: analyse the rest cases
# most of this comes from the definitition[]

In [25]:
df_unlabeled = df_clean[~df_clean['masked_plain_text_def'].str.contains('B_MATH_TERM')]
df_unlabeled['extraction_rule'].value_counts()

definition[]              2006
definition + \emph{}       124
theorem + \emph{}           31
definition + \textit{}      19
theorem + \textit{}          5
Name: extraction_rule, dtype: int64

In [26]:
df_clean = df_clean[df_clean['masked_plain_text_def'].str.contains('B_MATH_TERM')]

In [27]:
df_clean['extraction_rule'].value_counts()

definition + \emph{}      10991
definition + \textit{}     1920
definition[]               1333
theorem + \emph{}           886
theorem + \textit{}         157
Name: extraction_rule, dtype: int64

In [28]:
df_clean['labeled_def'] = df_clean['masked_plain_text_def'].apply(label_text)

In [29]:
df_clean['plain_text_term'].value_counts()[:60]

                                       52
equivalent                             31
<cit.>                                 20
admissible                             20
balanced                               20
weight                                 20
regular                                18
compatible                             18
simple                                 16
minimal                                14
connected                              13
                                       13
composition                            13
rank                                   12
isomorphic                             12
symmetric                              12
dual                                   12
(<ref>                                 11
critical                               11
complete                               10
pure                                   10
degree                                 10
stable                                 10
closure                           

# Continue to clean the dataset...

In [30]:
df_clean[df_clean['plain_text_term'].str.match(r'^\s*(\([ivxlcdm]+\)\s*([,;]\s*\([ivxlcdm]+\)\s*)*)$')]

,term,def,file_name,extraction_rule,plain_text_def,plain_text_term,masked_plain_text_def,labeled_def
1386,(i) ;(ii),"\n\label{th11}If $G$ is a K\""{o}nig-Egerv\'{a}...",/home/shufan/que_tex/1506.00255.tex,theorem + \emph{},"\nIf G is a König-Egerváry graph, then\n\n(i) ...",(i) ;(ii),"\nIf G is a König-Egerváry graph, then\n\nB_MA...",O O O O O O O O O O O O O O O O O O O O O O O
1911,(ii);(iii),\n\label{CLDELPS}Let $X$ be a del Pezzo surfac...,/home/shufan/que_tex/One_Sing.tex,theorem + \emph{},\nLet X be a del Pezzo surface of degree d:=(X...,(ii);(iii),\nLet X be a del Pezzo surface of degree d:=(X...,O O O O O O O O O O O O O O O O O O O O O O O ...
3015,(i);(ii);(iii);(iv),\n\label{th15}Let $\Gamma\subseteq\Omega(G)$ a...,/home/shufan/que_tex/1512.01994.tex,theorem + \emph{},"\nLet Γ⊆Ω(G) and ∅≠Γ^'⊆MaxCritIndep(G), be suc...",(i);(ii);(iii);(iv),"\nLet Γ⊆Ω(G) and ∅≠Γ^'⊆MaxCritIndep(G), be suc...",O O O O O O O O O O O O O O O O O O O O O O O ...
3802,(i);(ii);(iii);(iv),\n\label{th7}Let $G$ be a graph whose $\mathrm...,/home/shufan/que_tex/1506.00249.tex,theorem + \emph{},\nLet G be a graph whose core(G) is a critical...,(i);(ii);(iii);(iv),\nLet G be a graph whose core(G) is a critical...,O O O O O O O O O O O O B_MATH_TERM O B_MATH_T...
3803,(i);(ii) ;(iii);(iv),\n\label{perfect matching}If $\Gamma\subseteq\...,/home/shufan/que_tex/1506.00249.tex,theorem + \emph{},\nIf Γ⊆Ω( G) and\n|\n\n⋃\nΓ| +|\n\n⋂\nΓ| =2α(...,(i);(ii) ;(iii);(iv),\nIf Γ⊆Ω( G) and\n|\n\n⋃\nΓ| +|\n\n⋂\nΓ| =2α(...,O O O O O O O O O O O O B_MATH_TERM O O O O O ...
4727,(i) ;(ii);(iii),\n\label{th9}Let $A$ be a critical independent...,/home/shufan/que_tex/1102.1859.tex,theorem + \emph{},\nLet A be a critical independent set in a gra...,(i) ;(ii);(iii),\nLet A be a critical independent set in a gra...,O O O O O O O O O O O O O O O O O O B_MATH_TER...
5697,(i);(ii);(iii);(iv);(v);(vi) ;(vii),\n\label{Th33}Let $G$\ be a graph. Then the fo...,/home/shufan/que_tex/1102.1138.tex,theorem + \emph{},\nLet G be a graph. Then the following asserti...,(i);(ii);(iii);(iv);(v);(vi) ;(vii),\nLet G be a graph. Then the following asserti...,O O O O O O O O O O O B_MATH_TERM O O O B_MATH...
5698,(i) ;(ii);(iii),"\n\cite{Ore55}\label{th1} Let $G=\left( A,B,E...",/home/shufan/que_tex/1102.1138.tex,theorem + \emph{},"\n<cit.> Let G=( A,B,E). Then the following a...",(i) ;(ii);(iii),"\n<cit.> Let G=( A,B,E). Then the following a...",O O O O O O O O O O O O O O O O O O O O O O O ...
5699,(i) ;(ii) ;(iii);(iv);(v),"\n\label{Th4}Let $G=\left( A,B,E\right) $ be...",/home/shufan/que_tex/1102.1138.tex,theorem + \emph{},"\nLet G=( A,B,E) be a bipartite graph. Then t...",(i) ;(ii) ;(iii);(iv);(v),"\nLet G=( A,B,E) be a bipartite graph. Then t...",O O O O O O O O O O O O O O O O O O O O O O O ...
5700,(i) ;(ii);(iii),\n\label{th9}Let $X$ be a critical independent...,/home/shufan/que_tex/1102.1138.tex,theorem + \emph{},\nLet X be a critical independent set in a gra...,(i) ;(ii);(iii),\nLet X be a critical independent set in a gra...,O O O O O O O O O O O O O O O O O O B_MATH_TER...


In [31]:
df_clean = df_clean[~df_clean['plain_text_term'].str.match(r'^\s*(\([ivxlcdm]+\)\s*([,;]\s*\([ivxlcdm]+\)\s*)*)$')]

In [32]:
is_empty_or_only_whitespaces = df_clean['plain_text_term'].str.isspace() | df_clean['plain_text_term'].str.len() == 0
df_clean[is_empty_or_only_whitespaces]

,term,def,file_name,extraction_rule,plain_text_def,plain_text_term,masked_plain_text_def,labeled_def
129,\Csimple,Let $P$ be a polyhedron. We say $P$ is \Csimpl...,/home/shufan/que_tex/oncircuits.tex,definition[],Let P be a polyhedron. We say P is if all cir...,,Let P be a polyhedron. We say P is B_MATH_TERM...,O O O O O O O O O B_MATH_TERM O O O O O O O O ...
1033,\Cyclewidth,Let $D$ be a \digraph.\n\t\tA cycle decomposit...,/home/shufan/que_tex/bipgrid.tex,definition[],Let D be a .\n\t\tA cycle decomposition of D i...,,Let D be a .\n\t\tA cycle decomposition of D i...,O O O O O O O O O O O O O O O O O O O O O O O ...
1045,\cycPoros,[\Cyclewidth]\n\t\tLet $D$ be a \digraph.\n\t\...,/home/shufan/que_tex/bipgrid.tex,definition + \emph{},[]\n\t\tLet D be a .\n\t\tA cycle decompositio...,,[]\n\t\tLet D be a .\n\t\tA cycle decompositio...,O O O O O O O O O O O O O O O O O O O O O O O ...
1047,\matPoros,[matching-porosity]\n\t\tLet $G$ be a matching...,/home/shufan/que_tex/bipgrid.tex,definition + \emph{},[matching-porosity]\n\t\tLet G be a matching c...,,[matching-porosity]\n\t\tLet G be a matching c...,O O O O O O O O O O O O O B_MATH_TERM O O O O O
1598,\edg,[non-boundary edge classes]\label{def:non-boun...,/home/shufan/que_tex/trilongest.tex,definition + \emph{},[non-boundary edge classes] Let _π be an M-bou...,,[non-boundary edge classes] Let _π be an M-bou...,O O O O O O O O O O O O O O O O O O O O O O O ...
1599,\edg,[forward extension]\label{def:forward-ext}\nLe...,/home/shufan/que_tex/trilongest.tex,definition + \emph{},[forward extension]\nLet _π be an M-bounded tr...,,[forward extension]\nLet _π be an M-bounded tr...,O O O O O O O O O O O O O O O O O O O O O O O ...
2066,\robust,Given two sets $A \subseteq L$ and $B \subsete...,/home/shufan/que_tex/eccc-arxiv.tex,definition[],"Given two sets A ⊆ L and B ⊆ R, we say that th...",,"Given two sets A ⊆ L and B ⊆ R, we say that th...",O O O O O O O O O O O O O O O O O O O O O O O ...
2127,$\EK$,Let $\K$ be a class of structures in $L_I := \...,/home/shufan/que_tex/Sokic_Pawliuk_amenable_au...,definition[],Let be a class of structures in L_I := {R_i :...,,Let be a class of structures in L_I := {R_i :...,O O O O O O O O O O O O O O O O O O O O O O O ...
2128,$\LEK$,Let $L_I := \{R_i : i \in I\}$ and $L_J := \{R...,/home/shufan/que_tex/Sokic_Pawliuk_amenable_au...,definition[],Let L_I := {R_i : i ∈ I} and L_J := {R_j : j ∈...,,Let L_I := {R_i : i ∈ I} and L_J := {R_j : j ∈...,O O O O O O O O O O O O O O O O O O O O O O O ...
2129,$\QOPstar$,"Let $(\K,\K^*)$ be an excellent pair. We say t...",/home/shufan/que_tex/Sokic_Pawliuk_amenable_au...,definition[],"Let (,^*) be an excellent pair. We say that ^*...",,"Let (,^*) be an excellent pair. We say that ^*...",O O O O O O O O O O O O B_MATH_TERM O O O O O ...


In [33]:
df_clean = df_clean[~is_empty_or_only_whitespaces]
df_clean

,term,def,file_name,extraction_rule,plain_text_def,plain_text_term,masked_plain_text_def,labeled_def
0,admissible $G$-cover,\label{def_admissibleGcover}\nLet $G$ be a fin...,/home/shufan/que_tex/abelian_tropical_covers.tex,definition + \emph{},\nLet G be a finite group and let X→ S be a fa...,admissible G-cover,\nLet G be a finite group and let X→ S be a fa...,O O O O O O O O O O O O O O O O O O O O O O O ...
3,light reduction pair;heavy reduction pair,"\n Let $\pi\in\fs_n$ and let $x,y$ be a pair ...",/home/shufan/que_tex/Post071005.tex,definition + \emph{},"\n Let π∈_n and let x,y be a pair of rooks th...",light reduction pair;heavy reduction pair,"\n Let π∈_n and let x,y be a pair of rooks th...",O O O O O O O O O O O O O O O O O O O O O O B_...
5,Local modification rule,\label{lacma} A \emph{local modification rule...,/home/shufan/que_tex/hereditary_submit.tex,definition[],"A local modification rule is a pair (A,T), w...",Local modification rule,A B_MATH_TERM I_MATH_TERM I_MATH_TERM is a p...,O B_MATH_TERM I_MATH_TERM I_MATH_TERM O O O O ...
7,Vertex sets,\label{vertset} A \emph{vertex set} is any se...,/home/shufan/que_tex/hereditary_submit.tex,definition[],A vertex set is any set which is at most cou...,Vertex sets,A vertex set is any set which is at most cou...,O O O O O O O O O O O O O O O O B_MATH_TERM O ...
8,Palettes,A \emph{finite palette} is a sequence $K = (K_...,/home/shufan/que_tex/hereditary_submit.tex,definition[],A finite palette is a sequence K = (K_j)_j=0^∞...,Palettes,A finite palette is a sequence K = (K_j)_j=0^∞...,O O O O O O O O O O O O O O O O O O O O O O O ...
...,...,...,...,...,...,...,...,...
17837,preferred direction;special claw corresponding...,\label{definition:SpecialClaw} Consider $\lang...,/home/shufan/que_tex/CosetsVoltagesAndDerivedE...,definition + \textit{},"Consider ⟨ G→ S, α→ A⟩, two adjacent vertices...",preferred direction;special claw corresponding...,"Consider ⟨ G→ S, α→ A⟩, two adjacent vertices...",O O O O O O O O O O O O O O O O O O O O O O O ...
17844,Cyclicity classes,\label{d:cyclasses}\nAccording to Theorem~\ref...,/home/shufan/que_tex/1307.3716.tex,definition[],"\nAccording to Theorem <ref>, ^d has exactly d...",Cyclicity classes,"\nAccording to Theorem <ref>, ^d has exactly d...",O O O O O O O O O O O O O O O O O O O O O O O ...
17845,Wielandt number,By {\em Wielandt number} we mean the following...,/home/shufan/que_tex/1307.3716.tex,definition[],By Wielandt number we mean the following funct...,Wielandt number,By B_MATH_TERM I_MATH_TERM we mean the followi...,O B_MATH_TERM I_MATH_TERM O O O O O O O O O O O O
17846,Girth,"\label{d:girth}\nThe {\em girth\/} of $\digr$,...",/home/shufan/que_tex/1307.3716.tex,definition[],"\nThe girth of , denoted it by g(),\nis the sm...",Girth,"\nThe B_MATH_TERM of , denoted it by g(),\nis ...",O B_MATH_TERM O O O O O O O O O O O O O O O O


In [34]:
df_clean = df_clean[~(df_clean['plain_text_term'].str.contains('�'))]
df_clean = df_clean[~(df_clean['masked_plain_text_def'].str.contains('�'))]

In [35]:
df_clean[df_clean['plain_text_term'].apply(lambda x: len(set(x)) == 1)]


,term,def,file_name,extraction_rule,plain_text_def,plain_text_term,masked_plain_text_def,labeled_def
109,$\vect{k,\nGiven a multigraded combinatorial Hopf algeb...,/home/shufan/que_tex/0910.5773.tex,definition + \emph{},\nGiven a multigraded combinatorial Hopf algeb...,k,\nGiven a multigraded combinatorial Hopf algeb...,O O O O O O O O O O O O O O O O O O O O O O O ...
110,$\vect{k,\nGiven a multigraded combinatorial Hopf algeb...,/home/shufan/que_tex/0910.5773.tex,definition + \emph{},\nGiven a multigraded combinatorial Hopf algeb...,k,\nGiven a multigraded combinatorial Hopf algeb...,O O O O O O O O O O O O O O O O O O O O O O O ...
477,$\mathcal{X,[$\mathcal{X}$-index] Let $\mathcal{X}$ be a c...,/home/shufan/que_tex/SIA-index.tex,definition + \emph{},[𝒳-index] Let 𝒳 be a class of matrices. The 𝒳-...,𝒳,[𝒳-index] Let 𝒳 be a class of matrices. The B_...,O O O O O O O O O O O O O O O O O O O O O O O ...
651,$\mathcal{A,"\nLet $L,U$ be subgroups of a group $G$ and $L...",/home/shufan/que_tex/1502.04615.tex,definition + \emph{},"\nLet L,U be subgroups of a group G and L be n...",𝒜,"\nLet L,U be subgroups of a group G and L be n...",O O O O O O O O O O O O O O O O O O O O O O O O
665,$\mathcal{E,\n If $\mathcal{W}$ is any vector subspace of ...,/home/shufan/que_tex/JournalVersion.tex,definition + \emph{},"\n If 𝒲 is any vector subspace of 𝒱, the ℰ-ann...",ℰ,"\n If 𝒲 is any vector subspace of 𝒱, the B_MAT...",O O O O O O O O O O O O O O O O O O O O O
1593,\edg;\edg;\edg,\label{thm:ladders-concentrate}\nLet $M\geq 3$...,/home/shufan/que_tex/trilongest.tex,theorem + \emph{},\nLet M≥ 3 and let 0<δ<1 be the constant from ...,;;,\nLet M≥ 3 and let 0<δ<1 be the constant from ...,O O O O O O O O O O O O O O O O O O O O O O O ...
1594,\edg;\edg;\edg;\edg;\edg;\edg,\label{def:ladders}\n [triangular ladders]\nL...,/home/shufan/que_tex/trilongest.tex,definition + \emph{},"\n [triangular ladders]\nLet Π⊂{ ,0,1}^* deno...",;;;;;,"\n [triangular ladders]\nLet Π⊂{B_MATH_TERM ,...",O O O O O O O O O O O O O O O O O O O O O O O ...
1595,\edg;\edg;\edg;\edg,"[$f$-fan]\label{def:fan}\nFor $ f \geq 3$, an ...",/home/shufan/que_tex/trilongest.tex,definition + \emph{},"[f-fan]\nFor f ≥ 3, an f-fan at some vertex a>...",;;;,"[f-fan]\nFor f ≥ 3, an f-fan at some vertex a>...",O O O O O O O O O O O O O O O O O O O O O O O ...
1596,\edg;\edg;\edg;\edg;\edg,\label{def:M-bnd}[$M$-bounded triangular ladde...,/home/shufan/que_tex/trilongest.tex,definition + \emph{},"[M-bounded triangular ladders]\nFor M≥3, defin...",;;;;,"[M-bounded triangular ladders]\nFor M≥3, defin...",O O O O O O O O O O O O O O O O O O O O O O O ...
1597,\edg;\edg;\edg;\edg;\edg;\edg,[boundary edges]\label{def:boundary}\nLet $ \s...,/home/shufan/que_tex/trilongest.tex,definition + \emph{},[boundary edges]\nLet _π be an M-bounded trian...,;;;;;,[boundary edges]\nLet _π be an M-bounded trian...,O O O O O O O O O O O O O O O O O O O O O O O ...


In [36]:
df_clean = df_clean[~df_clean['plain_text_term'].apply(lambda x: len(set(x)) == 1)]

In [37]:
import string
df_clean[df_clean['plain_text_term'].apply(lambda x: all(c not in string.ascii_letters for c in x))]

,term,def,file_name,extraction_rule,plain_text_def,plain_text_term,masked_plain_text_def,labeled_def
255,$\mathcal{C;color-class-minimal,"\label{def:cc-min}\n Let $X=(V,E)$ be a hyper...",/home/shufan/que_tex/1709.10063.tex,definition + \emph{},"\n Let X=(V,E) be a hypergraph with color cla...",𝒞;𝒸ℴ𝓁ℴ𝓇-𝒸𝓁𝒶𝓈𝓈-𝓂𝒾𝓃𝒾𝓂𝒶𝓁,"\n Let X=(V,E) be a hypergraph with color cla...",O O O O O O O O O O O O O O O O O O O O O O O ...
542,$\mathcal{U;$\mathcal{U,"\n Let $\mathcal{U} = (U_1, \dotsc, U_{p+q}...",/home/shufan/que_tex/CDMTaltogether7.tex,definition + \emph{},"\n Let 𝒰 = (U_1, …, U_p+q) be an ordered co...",𝒰;𝒰,"\n Let 𝒰 = (U_1, …, U_p+q) be an ordered co...",O O O O O O O O O O O O O O O O O O O O O O O ...
647,$\mathcal{C;Cayley scheme,"\nLet $G$ be a finite group, $\mathcal{R}$ a ...",/home/shufan/que_tex/1502.04615.tex,definition + \emph{},"\nLet G be a finite group, ℛ a family of bina...",𝒞;𝒞𝒶𝓎𝓁ℯ𝓎 𝓈𝒸𝒽ℯ𝓂ℯ,"\nLet G be a finite group, ℛ a family of bina...",O O O O O O O O O O O O O O O O O O O B_MATH_T...
1463,$\DISC(\delta)$,\label{def:DISC_delta}\n Let~$0<\delta\leq1$ ...,/home/shufan/que_tex/Fiedler.tex,definition[],\n Let 0<δ≤1 be given. We say that an n-vert...,(δ),\n Let 0<δ≤1 be given. We say that an n-vert...,O O O O O O O O O O O O O O B_MATH_TERM O O O ...
1464,$\DISC_2(\delta')$,\label{def:DISC_2_delta}\n Let~$0<\delta'\leq...,/home/shufan/que_tex/Fiedler.tex,definition[],\n Let 0<δ'≤1 be given. We say that an n-ver...,_2(δ'),\n Let 0<δ'≤1 be given. We say that an n-ver...,O O O O O O O O O O O O O O B_MATH_TERM O O O ...
1465,$\EIG(\eps)$,\label{def:EIG_epsilon}\n Let~$0<\eps\leq1$ b...,/home/shufan/que_tex/Fiedler.tex,definition[],\n Let 0<≤1 be given. We say that an n-verte...,(),\n Let 0<≤1 be given. We say that an n-verte...,O O O O O O O O O O O O O B_MATH_TERM O O O O ...
1467,"$\ZINTDISC(\rho;\eta,\sigma)$",\label{def:Z-INT-DISC}\n For positive reals $...,/home/shufan/que_tex/Fiedler.tex,definition[],"\n For positive reals η and σ, we say that A\...","(ρ;η,σ)","\n For positive reals η and σ, we say that A\...",O O O O O O O O O O O B_MATH_TERM O O O O O O ...
1468,"$\SINTDISC(\chia;\eta,\sigma)$",\label{def:S-INT-DISC}\n For positive reals $...,/home/shufan/que_tex/Fiedler.tex,definition[],"\n For positive reals η and σ, we say that A\...","(;η,σ)","\n For positive reals η and σ, we say that A\...",O O O O O O O O O O O B_MATH_TERM O O O O O O ...
1478,$2$-$\mathcal{M;$2$-$\mathcal{NM,\nLet $G$ be a $2$-edge-colored multigraph. We...,/home/shufan/que_tex/alternating-hamiltonian-c...,definition + \emph{},\nLet G be a 2-edge-colored multigraph. We say...,2-ℳ;2-𝒩ℳ,\nLet G be a 2-edge-colored multigraph. We say...,O O O O O O O O O O O O O O O O O O O O O O O ...
1507,:;1;;1;;2.,[\cite{LiPraeger}]\n\label{thm:LiPraeger}\nLet...,/home/shufan/que_tex/1905.00520.tex,theorem + \emph{},[<cit.>]\n\nLet G be a finite permutation grou...,:;1;;1;;2.,[<cit.>]\n\nLet G be a finite permutation grou...,O O O O O O O O O O O O O O O O O O O O O O O ...


In [38]:
df_clean = df_clean[~df_clean['plain_text_term'].apply(lambda x: all(c not in string.ascii_letters for c in x))]

In [39]:
# ex*, i.e., <cite>, i;ii

In [40]:
df_clean[ df_clean['plain_text_term'].str.match(r'^\s*([ivxlcd]+\s*([,;]\s*[ivxlcd]+\s*)*)$')] # remove m to not exlude 'mild'

,term,def,file_name,extraction_rule,plain_text_def,plain_text_term,masked_plain_text_def,labeled_def
3235,i;ii,\textbf{Laplacian test for fractals~\cite{Kiga...,/home/shufan/que_tex/Optimization_algorithm_on...,theorem + \emph{},Laplacian test for fractals <cit.>\n\n\t\nGive...,i;ii,Laplacian test for fractals <cit.>\n\n\t\nGive...,O O O O O O O O O O O O O O O O O O B_MATH_TER...
3238,i;ii,\n\t \n \n\noindent We define:\n\n\begin{item...,/home/shufan/que_tex/Optimization_algorithm_on...,definition + \emph{},\n\t \n \nWe define:\n\n \n\t\n\t\n\t\n i ...,i;ii,\n\t \n \nWe define:\n\n \n\t\n\t\n\t\n B_...,O O B_MATH_TERM O O O O O O O O O O B_MATH_TER...
3241,i;ii,{\textbf{Extrema }}\\\n\t\n\noindent Given a c...,/home/shufan/que_tex/Optimization_algorithm_on...,definition + \emph{},Extrema \n\n\t\nGiven a continuous function u ...,i;ii,Extrema \n\n\t\nGiven a continuous function u ...,O O O O O O O O O O O O O O O O O O O O O B_MA...
8839,c;c;c;c;c;c,\label{def:A}\nLet $E$ be a heap and let \n\b...,/home/shufan/que_tex/ArXThesis.tex,definition + \emph{},"\nLet E be a heap and let \n\nc = {α_1, α_2, ...",c;c;c;c;c;c,\nLet E be a heap and let \n\nB_MATH_TERM = {...,O O O O O O O B_MATH_TERM O O O O O O O O O O ...
9113,i;i,"\nLet $P_m, P_n$ be a path of order $m, n \ge ...",/home/shufan/que_tex/1402.0615.tex,theorem + \textit{},"\nLet P_m, P_n be a path of order m, n ≥ 2. Th...",i;i,"\nLet P_m, P_n be a path of order m, n ≥ 2. Th...",O O O O O O O O O O O O O O O O O O O O O O O ...
9114,i;i;i;i;i,\nLet $C_m$ be a cycle of order $m \ge3$ and $...,/home/shufan/que_tex/1402.0615.tex,theorem + \textit{},\nLet C_m be a cycle of order m ≥3 and P_n be ...,i;i;i;i;i,\nLet C_m be a cycle of order m ≥3 and P_n be ...,O O O O O O O O O O O O O O O O O O O O O O O ...
9115,i;i,\nLet $S_m$ be the star graph of order $m \ge ...,/home/shufan/que_tex/1402.0615.tex,theorem + \textit{},\nLet S_m be the star graph of order m ≥ 2 and...,i;i,\nLet S_m be the star graph of order m ≥ 2 and...,O O O O O O O O O O O O O O O O O O O O O O O ...
9116,i;i;i;i,\nLet $K_m$ be a\ncomplete graph of order $m \...,/home/shufan/que_tex/1402.0615.tex,theorem + \textit{},\nLet K_m be a\ncomplete graph of order m ≥ 2 ...,i;i;i;i,\nLet K_m be a\ncomplete graph of order m ≥ 2 ...,O O O O O O O O O O O O O O O O O O O O O O O ...
9117,i;i,\nLet $W_m$ be a wheel graph of\norder $m \ge ...,/home/shufan/que_tex/1402.0615.tex,theorem + \textit{},\nLet W_m be a wheel graph of\norder m ≥ 3 and...,i;i,\nLet W_m be a wheel graph of\norder m ≥ 3 and...,O O O O O O O O O O O O O O O O O O O O O O O ...
9118,i;i;i;i;i;i;i;i;i;i;i,"\n Let $C_n, C_m$ be a cycle of order $n, m \g...",/home/shufan/que_tex/1402.0615.tex,theorem + \textit{},"\n Let C_n, C_m be a cycle of order n, m ≥ 3. ...",i;i;i;i;i;i;i;i;i;i;i,"\n Let C_n, C_m be a cycle of order n, m ≥ 3. ...",O O O O O O O O O O O O O O O O O O O O O O O ...


In [41]:
df_clean = df_clean[~df_clean['plain_text_term'].str.match(r'^\s*([ivxlcd]+\s*([,;]\s*[ivxlcd]+\s*)*)$')]

In [42]:
df_clean[ df_clean['plain_text_term'].str.match(r'^\s*([i.e]+\s*([,;]\s*[i.e]+\s*)*)$')]

,term,def,file_name,extraction_rule,plain_text_def,plain_text_term,masked_plain_text_def,labeled_def
5767,i.e.;i.e.,[\cite{MihalceaAdvances}]\label{T:OneBoxPlusPi...,/home/shufan/que_tex/arXivPost3_24_14.tex,theorem + \textit{},"[<cit.>]\nLet Λ = [t_1, …, t_n]. Suppose that...",i.e.;i.e.,"[<cit.>]\nLet Λ = [t_1, …, t_n]. Suppose that...",O O O O O O O O O O O O O O O O O O O O O O O ...
7360,i.e.,"\n$S(n,3)$ has Lex nested solutions for $EIP$\...",/home/shufan/que_tex/eiponsn3a.tex,theorem + \textit{},"\nS(n,3) has Lex nested solutions for EIP\n\n\...",i.e.,"\nS(n,3) has Lex nested solutions for EIP\n\n\...",O O O O O O O O O O O O O O O O O O O O O O O ...
10816,i.e.,\label{Thm:DLW}\n\tConsider the following syst...,/home/shufan/que_tex/ClassesNonCloses.tex,theorem + \emph{},\n\tConsider the following system:\n\t\n <r...,i.e.,\n\tConsider the following system:\n\t\n <r...,O O O O O O O O O O O O O O O O O O O O O O O ...


In [43]:
df_clean = df_clean[~df_clean['plain_text_term'].str.match(r'^\s*([i.e]+\s*([,;]\s*[i.e]+\s*)*)$')]

In [44]:
df_clean[ df_clean['plain_text_term'].str.match(r'^\s*([ex.]+\s*([,;]\s*[ex.]+\s*)*)$')]

,term,def,file_name,extraction_rule,plain_text_def,plain_text_term,masked_plain_text_def,labeled_def
1856,ex,\cite{erdHos1966}\label{p+1 chromatic}\nIf $\m...,/home/shufan/que_tex/1903.10319.tex,theorem + \emph{},<cit.>\nIf ℒ is a family of graphs with subchr...,ex,<cit.>\nIf ℒ is a family of graphs with subchr...,O O O O O O O O O O O O O O O
11684,ex;ex;ex,\n \label{thm:SMain}\nIf $P$ is chosen unifo...,/home/shufan/que_tex/0810.3670.tex,theorem + \emph{},\n \nIf P is chosen uniformly at random from...,ex;ex;ex,\n \nIf P is chosen uniformly at random from...,O O O O O O O O O O O O O O O O O O O O O O O ...
11685,ex;ex,\label{thm:convavg}\nIf $P$ is chosen uniform...,/home/shufan/que_tex/0810.3670.tex,theorem + \emph{},"\nIf P is chosen uniformly at random from _0,...",ex;ex,"\nIf P is chosen uniformly at random from _0,...",O O O O O O O O O O O O O O O O O O O O O O


In [45]:
df_clean = df_clean[~df_clean['plain_text_term'].str.match(r'^\s*([ex.]+\s*([,;]\s*[ex.]+\s*)*)$')]

In [46]:
#if, for, or
df_clean[ df_clean['plain_text_term'].str.match(r'^\s*([ifor]+\s*([,;]\s*[ifor]+\s*)*)$')]

,term,def,file_name,extraction_rule,plain_text_def,plain_text_term,masked_plain_text_def,labeled_def
4697,if;if,"\label{THM:mod 2}\nFor all $n\geq0$,\n\begin{a...",/home/shufan/que_tex/1805.06575.tex,theorem + \emph{},"\nFor all n≥0,\n\n M^*(0,2,n) >M^*(1,2,n...",if;if,"\nFor all n≥0,\n\n M^*(0,2,n) >M^*(1,2,n...",O O O O O B_MATH_TERM O O O O O B_MATH_TERM O O O
4698,if;if,"\label{THM:mod 3}\nFor all $n\geq0$,\n\begin{a...",/home/shufan/que_tex/1805.06575.tex,theorem + \emph{},"\nFor all n≥0,\n\n M^*(0,3,n) >M^*(1,3,n)...",if;if,"\nFor all n≥0,\n\n M^*(0,3,n) >M^*(1,3,n)...",O O O O O B_MATH_TERM O O O O O B_MATH_TERM O ...
4699,if;if;if,"\label{th:main-asy}\nFor $n\ge 1$,\n\begin{equ...",/home/shufan/que_tex/1805.06575.tex,theorem + \emph{},"\nFor n≥ 1,\n\n M^*(0,3,n)-M^*(1,3,n)=c(n) ...",if;if;if,"\nFor n≥ 1,\n\n M^*(0,3,n)-M^*(1,3,n)=c(n) ...",O O O O O O O B_MATH_TERM O O O O B_MATH_TERM ...
4700,if;if,"\label{THM:mod 4}\nFor all $n\geq0$,\n\begin{a...",/home/shufan/que_tex/1805.06575.tex,theorem + \emph{},"\nFor all n≥0,\n\n M^*(0,4,n) >M^*(2,4,n)...",if;if,"\nFor all n≥0,\n\n M^*(0,4,n) >M^*(2,4,n)...",O O O O O B_MATH_TERM O O O O O B_MATH_TERM O ...
4741,if;or;if,\n\label{Th8}\nLet $ T $ be a tree with $ c(T)...,/home/shufan/que_tex/Inertia.tex,theorem + \textit{},\n\nLet T be a tree with c(T)≥ 1.\nThen\n\n ...,if;or;if,\n\nLet T be a tree with c(T)≥ 1.\nThen\n\n ...,O O O O O O O O O O O B_MATH_TERM O B_MATH_TER...
4990,iff,"\n\label{[x]def} \cite{ESS}, 3.2. Let $(X,r)$ ...",/home/shufan/que_tex/CYBE30507JSC.tex,definition + \emph{},"\n <cit.>, 3.2. Let (X,r) be a nondegenerate\n...",iff,"\n <cit.>, 3.2. Let (X,r) be a nondegenerate\n...",O O O O O O O O O O O O O O O O O O O O O B_MA...
4996,iff,By $\Gamma(x)$ we denote the\nsubgraph of (th...,/home/shufan/que_tex/CYBE30507JSC.tex,definition + \emph{},By Γ(x) we denote the\nsubgraph of (the\ncomp...,iff,By Γ(x) we denote the\nsubgraph of (the\ncomp...,O O O O O O O O O O O O O O O O O O O O O O O ...
5656,if ;if ;if,"\label{Thm:mod 5}\nFor all $n\geq0$,\n\begin{a...",/home/shufan/que_tex/1706.08223.tex,theorem + \emph{},"\nFor all n≥0,\n\n w_t(5n+3)≡ w_t(5n+4)≡ 05...",if ;if ;if,"\nFor all n≥0,\n\n w_t(5n+3)≡ w_t(5n+4)≡ 05...",O O O O O O B_MATH_TERM O O O O B_MATH_TERM O ...
5657,if ;if ;if,\label{weighted analog thm }\nFor all $n\geq0$...,/home/shufan/que_tex/1706.08223.tex,theorem + \emph{},"\nFor all n≥0,\n\n c_t(5n+3)= c_t(5n+4)= 0,...",if ;if ;if,"\nFor all n≥0,\n\n c_t(5n+3)= c_t(5n+4)= 0,...",O O O O O O B_MATH_TERM O O O O B_MATH_TERM O ...
5688,for;for;for,"\label{thm:alternate thm}\nFor $n\geq0$,\n\beg...",/home/shufan/que_tex/1705.10067.tex,theorem + \emph{},"\nFor n≥0,\n\n M_k(0,2,2n) >M_k(1,2,2n),...",for;for;for,"\nFor n≥0,\n\n M_k(0,2,2n) >M_k(1,2,2n),...",O O O O B_MATH_TERM O O O B_MATH_TERM O O O O ...


In [47]:
df_clean = df_clean[~ df_clean['plain_text_term'].str.match(r'^\s*([iforIFOR]+\s*([,;]\s*[iforIFOR]+\s*)*)$')]

In [48]:
df_clean[ df_clean['plain_text_term'].str.match(r'^\s*([IEXiex.]+\s*([,;]\s*[IEXiex.]+\s*)*)$')]

,term,def,file_name,extraction_rule,plain_text_def,plain_text_term,masked_plain_text_def,labeled_def
4185,i.e.;I.e.,[Pattern Theorem;~\cite{Bender1992}]\n Suppos...,/home/shufan/que_tex/knotasymp_paper.tex,theorem + \textit{},[Pattern Theorem; <cit.>]\n Suppose all of th...,i.e.;I.e.,[Pattern Theorem; <cit.>]\n Suppose all of th...,O O O O O O O O O O O O O O O O B_MATH_TERM O ...
14521,I.e.,\nPOSET is the category (see \cite{Mac}) whose...,/home/shufan/que_tex/TheRangeofaStOp_revisionf...,definition + \textit{},\nPOSET is the category (see <cit.>) whose obj...,I.e.,\nPOSET is the category (see <cit.>) whose obj...,O O O O O O O O O O O O O O O O O O O O O B_MA...


In [49]:
df_clean = df_clean[~df_clean['plain_text_term'].str.match(r'^\s*([IEXiex.]+\s*([,;]\s*[IEXiex.]+\s*)*)$')]

In [51]:
df_clean.to_csv('cleaned_plain-text_term_def_labeled.csv',index=False)

In [52]:
df_clean = pd.read_csv('cleaned_plain-text_term_def_labeled.csv', delimiter=',')


## update: from "\n A graph 𝐺 is \emph{non}-𝑘-equivalent if..." the term should be \emph{non}-𝑘-equivalent

In [53]:
combi_pattern = r'(\\emph\{.*?\}-)|(\\textit\{.*?\}-)'

df_clean[df_clean['def'].str.contains(combi_pattern)]

/tmp/ipykernel_19300/2589086422.py:3: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df_clean[df_clean['def'].str.contains(combi_pattern)]


,term,def,file_name,extraction_rule,plain_text_def,plain_text_term,masked_plain_text_def,labeled_def
316,Statement of Theorem \ref{thm:singleton_b_is_a...,\n%\label{thm:singleton_b_is_a_prime}\n\emph{S...,/home/shufan/que_tex/1803.03954.tex,theorem + \emph{},\n\nStatement of Theorem <ref>: Let n be a pos...,Statement of Theorem <ref>,\n\nB_MATH_TERM I_MATH_TERM I_MATH_TERM I_MATH...,B_MATH_TERM I_MATH_TERM I_MATH_TERM O O O O O ...
322,weak descent composition of $\rho$,"[\cite{Ass-T}]\n For a reduced word $\rho$, d...",/home/shufan/que_tex/weak_EG.tex,definition + \emph{},"[<cit.>]\n For a reduced word ρ, define the w...",weak descent composition of ρ,"[<cit.>]\n For a reduced word ρ, define the B...",O O O O O O O O B_MATH_TERM I_MATH_TERM I_MATH...
328,weak descent composition of $T$,[\cite{Ass-W}]\n For a standard key tableau $...,/home/shufan/que_tex/weak_EG.tex,definition + \emph{},"[<cit.>]\n For a standard key tableau T, defi...",weak descent composition of T,"[<cit.>]\n For a standard key tableau T, defi...",O O O O O O O O O B_MATH_TERM I_MATH_TERM I_MA...
702,"\{1,2,3\;basic matroids","\label{def:decomposition}\nA \emph{\{1,2,3\}-...",/home/shufan/que_tex/Decomposition.tex,definition + \emph{},"\nA {1,2,3}-decomposition of a matroid M̃ is ...","{1,2,3 basic matroids",\nA B_MATH_TERM-decomposition of a matroid M̃...,O O O O O O O O O O O O O O B_MATH_TERM I_MATH...
731,NP,\label{t:CTC}\n{\sc Common-Tree-Containment} i...,/home/shufan/que_tex/write-up.tex,theorem + \emph{},\nCommon-Tree-Containment is NP-complete when ...,NP,\nCommon-Tree-Containment is B_MATH_TERM-compl...,O O O O O O O O O O O O
...,...,...,...,...,...,...,...,...
14485,ME,\label{DefMEequivalence}\nSuppose $\Sigma$ is ...,/home/shufan/que_tex/1807.06171.tex,definition + \textit{},"\nSuppose Σ is an ordered alphabet and w,w'∈Σ^...",ME,"\nSuppose Σ is an ordered alphabet and w,w'∈Σ^...",O O O O O O O O O O O O O O O O O O O O O O O ...
14523,directed path decomposition;bags;width,"\n\tLet $D$ be a digraph, $P$ a directed path ...",/home/shufan/que_tex/DMW.tex,definition + \emph{},"\n\tLet D be a digraph, P a directed path and ...",directed path decomposition;bags;width,"\n\tLet D be a digraph, P a directed path and ...",O O O O O O O O O O O O O O O O O O O O O O O ...
14655,lattice path,\nA \emph{lattice path} of length $n$ is a se...,/home/shufan/que_tex/1112.4026.tex,definition + \emph{},\nA lattice path of length n is a sequence (γ...,lattice path,\nA B_MATH_TERM I_MATH_TERM of length n is a ...,O B_MATH_TERM I_MATH_TERM O O O O O O O O O O ...
14740,MSE,"\label{StrongMButNotMSE}\nSuppose $\Sigma=\{a,...",/home/shufan/que_tex/1702.03647.tex,theorem + \textit{},"\nSuppose Σ={a,b,c} and w,w'∈Σ^*. Let w=w_0β_1...",MSE,"\nSuppose Σ={a,b,c} and w,w'∈Σ^*. Let w=w_0β_1...",O O O O O O O O O O O O O O O O O O O O O O O ...


In [55]:
def mask_emph_and_tail(text):
    pattern = r'\\emph\{(.*?)\}(-\S*)?'

    def replace(match):
        words = match.group(1).split()
        if not words:
            return ''
        masked_words = ['B_MATH_TERM'] + ['I_MATH_TERM'] * (len(words) - 1)
        return ' '.join(masked_words)
    return re.sub(pattern, replace, text)

# Example usage
text = r'This is some text with \emph{emphasized words A B C D} in it. \emph{blah} . \emph{blah blah}-mask_me, please. \n A graph 𝐺 is \emph{non}-𝑘-equivalent if...'
masked_text = mask_emph_and_tail(text)
print(masked_text)

This is some text with B_MATH_TERM I_MATH_TERM I_MATH_TERM I_MATH_TERM I_MATH_TERM I_MATH_TERM in it. B_MATH_TERM . B_MATH_TERM I_MATH_TERM please. \n A graph 𝐺 is B_MATH_TERM if...


In [56]:
def mask_textit_and_tail(text):
    pattern = r'\\textit\{(.*?)\}(-\S*)?'

    def replace(match):
        words = match.group(1).split()
        if not words:
            return ''
        masked_words = ['B_MATH_TERM'] + ['I_MATH_TERM'] * (len(words) - 1)
        return ' '.join(masked_words)
    return re.sub(pattern, replace, text)

# Example usage
text = r'This is some text with \textit{emphasized words A B C D} in it. \textit{blah}. \textit{blah blah}-mask_me, please. \n A graph 𝐺 is \textit{non}-𝑘-equivalent if...'
masked_text = mask_textit_and_tail(text)
print(masked_text)

This is some text with B_MATH_TERM I_MATH_TERM I_MATH_TERM I_MATH_TERM I_MATH_TERM I_MATH_TERM in it. B_MATH_TERM. B_MATH_TERM I_MATH_TERM please. \n A graph 𝐺 is B_MATH_TERM if...


In [57]:
def mask_def_sentence_update_tail(sent, extraction_rule, masked_plain_text_def, term=''):
    combi_pattern = r'(\\emph\{.*?\}-)|(\\textit\{.*?\}-)'

    if re.search(combi_pattern, sent):
        if 'emph' in extraction_rule:
            masked_text = mask_emph_and_tail(sent)
            return LatexNodes2Text().latex_to_text(masked_text) # convert to plain text
        elif 'textit' in extraction_rule:
            masked_text = mask_textit_and_tail(sent)
            return LatexNodes2Text().latex_to_text(masked_text) # convert to plain text
        else:
            return masked_plain_text_def
    else:
        return masked_plain_text_def
    

df_clean['masked_plain_text_def'] = df_clean.apply(lambda x: mask_def_sentence_update_tail(x['def'], x['extraction_rule'], x['masked_plain_text_def'], x['term']), axis=1)

In [58]:
df_clean['labeled_def'] = df_clean['masked_plain_text_def'].apply(label_text)

In [59]:
def extract_textit_combi_terms(text):
    pattern = r'\\textit\{(.*?)\}(-\S*)?'
    matches = re.findall(pattern, text)
    result = []
    for match in matches:
        result.append(''.join(match))
    return ';'.join(result)
# Example usage
text = r'\textit{blah blah}! This is some text with \textit{emphasized words A B C D}, in it. \textit{blah}. \textit{blah blah}-bibi is a combi. '
masked_text = extract_textit_combi_terms(text)
print(masked_text)

blah blah;emphasized words A B C D;blah;blah blah-bibi


In [60]:
def extract_emph_combi_terms(text):
    pattern = r'\\emph\{(.*?)\}(-\S*)?'
    matches = re.findall(pattern, text)
    result = []
    for match in matches:
        result.append(''.join(match))
    return ';'.join(result)
# Example usage
text = r'\emph{blah blah}! This is some text with \emph{emphasized words A B C D}, in it. \emph{blah}. \emph{blah blah}-bibi is a combi. '
masked_text = extract_emph_combi_terms(text)
print(masked_text)

blah blah;emphasized words A B C D;blah;blah blah-bibi


In [61]:
def update_combi_terms(sent, extraction_rule, term):
    combi_pattern = r'(\\emph\{.*?\}-)|(\\textit\{.*?\}-)'
    if re.search(combi_pattern, sent):
        if 'emph' in extraction_rule:
            combi_term = extract_emph_combi_terms(sent)
            return combi_term
        elif 'textit' in extraction_rule:
            combi_term = extract_textit_combi_terms(sent)
            return combi_term
        else:
            return term
    else:
        return term
    

df_clean['term'] = df_clean.apply(lambda x: update_combi_terms(x['def'], x['extraction_rule'], x['term']), axis=1)

In [62]:
def update_plain_text_combi_terms(sent, term, plain_text_term):
    combi_pattern = r'(\\emph\{.*?\}-)|(\\textit\{.*?\}-)'
    if re.search(combi_pattern, sent):
        return LatexNodes2Text().latex_to_text(term)
    else:
        return plain_text_term
    

df_clean['plain_text_term'] = df_clean.apply(lambda x: update_plain_text_combi_terms(x['def'], x['term'], x['plain_text_term']), axis=1)

In [63]:
df_clean[df_clean['def'].str.contains(combi_pattern)].to_csv("combis_0717.csv")

/tmp/ipykernel_19300/2504376093.py:1: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df_clean[df_clean['def'].str.contains(combi_pattern)].to_csv("combis_0717.csv")


In [64]:
# no
df_clean[ df_clean['plain_text_term'].str.match(r'^\s*([noNO]+\s*([,;]\s*[noNO]+\s*)*)$')]

,term,def,file_name,extraction_rule,plain_text_def,plain_text_term,masked_plain_text_def,labeled_def
7448,no,[Schwenk]\label{t:S} Let $1\leq m\leq n$. The ...,/home/shufan/que_tex/Knight.tex,theorem + \emph{},[Schwenk] Let 1≤ m≤ n. The m× n\n chessboard ...,no,[Schwenk] Let 1≤ m≤ n. The m× n\n chessboard ...,O O O O O O O O O O B_MATH_TERM O O O O O O O ...
7449,no,[DeMaio and Mathew]\label{t:DeM} Let $2\leq m\...,/home/shufan/que_tex/Knight.tex,theorem + \emph{},[DeMaio and Mathew] Let 2≤ m≤ n≤\n p. The m×...,no,[DeMaio and Mathew] Let 2≤ m≤ n≤\n p. The m×...,O O O O O O O O O O O O O O B_MATH_TERM O O O ...
7450,no,\label{t:main}\nLet $2\leq n_1\leq n_2\leq... ...,/home/shufan/que_tex/Knight.tex,theorem + \emph{},"\nLet 2≤ n_1≤ n_2≤... ≤ n_k, with k≥ 3. The\nn...",no,"\nLet 2≤ n_1≤ n_2≤... ≤ n_k, with k≥ 3. The\nn...",O O O O O O O O O O O O O O B_MATH_TERM O O O ...
7645,n;n;n;n,"(\cite[Proposition 4, 5, 6, 7, 8, 9 $\&$ 10]{...",/home/shufan/que_tex/1402.0615.tex,theorem + \textit{},(<cit.>) \n\n\n * Let P_n be a path of the s...,n;n;n;n,(<cit.>) \n\n\n * Let P_n be a path of the s...,O O O O O O O O O O O O O O O O O O O O O O O ...


In [65]:
df_clean = df_clean[ ~df_clean['plain_text_term'].str.match(r'^\s*([noNO]+\s*([,;]\s*[noNO]+\s*)*)$')]

In [66]:
# (1) (i) (TB)
df_clean[df_clean['plain_text_term'].str.match(r'^\s*(\([A-Za-z0-9]+\)\s*([,;]\s*\([A-Za-z0-9]+\)\s*)*)$')]

,term,def,file_name,extraction_rule,plain_text_def,plain_text_term,masked_plain_text_def,labeled_def
192,"(P2), (P3), (P4)",\label{Correctness of ASE}\nThe association sc...,/home/shufan/que_tex/Extensibility.tex,theorem + \textit{},\nThe association scheme extension algorithm w...,"(P2), (P3), (P4)",\nThe association scheme extension algorithm w...,O O O O O O O O O O O O O O O O O O O O O O O ...
293,$\Comp{(n)}$,Let $\Comp(n)$ denote the poset of ordered int...,/home/shufan/que_tex/alpha_colored_may_27.tex,definition[],Let (n) denote the poset of ordered integer pa...,(n),Let (n) denote the poset of ordered integer pa...,O O O O O O O O O O O O O O O O O O O O O O O ...
803,(TB);(i);(ii),[Kriesell~\cite{Kriesell:1325227}] \label{T5}\...,/home/shufan/que_tex/double-col-critical_prepr...,theorem + \emph{},[Kriesell <cit.>] \n A graph is a minimal cri...,(TB);(i);(ii),[Kriesell <cit.>] \n A graph is a minimal cri...,O O O O O O O O O O O O O O O O O O O O O O O ...
3140,(a);(b);(c),"\label{th:logconc-a}\nSuppose $x\in(0,1)$ is f...",/home/shufan/que_tex/1509.05120.tex,theorem + \emph{},"\nSuppose x∈(0,1) is fixed. Then:\n\n(a) if 0<...",(a);(b);(c),"\nSuppose x∈(0,1) is fixed. Then:\n\nB_MATH_TE...",O O O O O B_MATH_TERM O O O O O O O O O O B_MA...
4245,(a);(b),\label{thm:foulkes}\n \emph{(a)} Let $\mu\vda...,/home/shufan/que_tex/submission.tex,theorem + \emph{},"\n (a) Let μ⊢ n, where n=2k+1. Then\n \n ...",(a);(b),"\n B_MATH_TERM Let μ⊢ n, where n=2k+1. Then\n...",B_MATH_TERM O O O O O O O O O O O O O O O O O ...
4246,(a);(b);(c);(d),\label{thm:bn}\n\emph{(a)} If $n$ is odd then...,/home/shufan/que_tex/submission.tex,theorem + \emph{},\n(a) If n is odd then\n \n b(n) = b^∗(n)...,(a);(b);(c);(d),\nB_MATH_TERM If n is odd then\n \n b(n) ...,B_MATH_TERM O O O O O O O O O O O B_MATH_TERM ...
7355,(a);(i);(ii);(b),\label{IB_Freezing}\n{Let $k\geq 3$. There exi...,/home/shufan/que_tex/1808.04060.tex,theorem + \emph{},\nLet k≥ 3. There exists 0<ϵ_q=o_q(1) and pos...,(a);(i);(ii);(b),\nLet k≥ 3. There exists 0<ϵ_q=o_q(1) and pos...,O O O O O O O O O O O O O O O O O O O O O O O ...
8059,(b);(c),\n\label{grids}\n\n\begin{enumerate}\n\item[(a...,/home/shufan/que_tex/ProtectionSurvey_Klosterm...,theorem + \emph{},\n\n\n\n\n (a) τ_m^∞(P_1 P_n)=n-1.\n\n\n\n ...,(b);(c),\n\n\n\n\n (a) τ_m^∞(P_1 P_n)=n-1.\n\n\n\n ...,O O O B_MATH_TERM O O O O O O O O O B_MATH_TER...
9041,(Furstenberg),\emph{(Furstenberg)}\n\label{furst1}\nLet $\ka...,/home/shufan/que_tex/1505.01379.tex,theorem + \emph{},(Furstenberg)\n\nLet κ be a field of positive ...,(Furstenberg),B_MATH_TERM\n\nLet κ be a field of positive ch...,B_MATH_TERM O O O O O O O O O O O O O O O O O ...
9042,(Furstenberg),\emph{(Furstenberg)}\n\label{furst2}\nLet $\ma...,/home/shufan/que_tex/1505.01379.tex,theorem + \emph{},(Furstenberg)\n\nLet 𝔽_q be a finite field of ...,(Furstenberg),B_MATH_TERM\n\nLet 𝔽_q be a finite field of ch...,B_MATH_TERM O O O O O O O O O O O O O O O O O ...


In [67]:
# (1) (i) (TB)
df_clean = df_clean[ ~df_clean['plain_text_term'].str.match(r'^\s*(\([A-Za-z0-9]+\)\s*([,;]\s*\([A-Za-z0-9]+\)\s*)*)$')]

In [68]:
df_clean.to_csv('cleaned_plain-text_term+combined-term_def_labeled.csv',index=False)

In [69]:
# citations
df_clean[df_clean['plain_text_term'].str.contains('<cit')]

,term,def,file_name,extraction_rule,plain_text_def,plain_text_term,masked_plain_text_def,labeled_def
98,"\textbf{(Guth, Katz, \cite[Theorem 4.1]{Guth_K...","\emph{\textbf{(Guth, Katz, \cite[Theorem 4.1]{...",/home/shufan/que_tex/1408.5867.tex,theorem + \emph{},"(Guth, Katz, <cit.> ) Let 𝔊 be a finite set of...","(Guth, Katz, <cit.>",B_MATH_TERM I_MATH_TERM I_MATH_TERM I_MATH_TER...,B_MATH_TERM I_MATH_TERM I_MATH_TERM I_MATH_TER...
592,"\cite[Tab.\,3]{BDFKMP-ArXiv2015Greedy;2;A.;(i)...","\label{th1_main}\nLet $t_{2}(2,q)$ be the smal...",/home/shufan/que_tex/BDFKMP_TabSmallArcFOP4300...,theorem + \emph{},"\nLet t_2(2,q) be the smallest size of a compl...",<cit.>,"\nLet t_2(2,q) be the smallest size of a compl...",O O O O O O O O O O O O O O O O O O O O O O O ...
891,(\cite[Theorem~3.1]{KRS,\emph{(\cite[Theorem~3.1]{KRS} )}\label{THkrse...,/home/shufan/que_tex/1512.06393.tex,theorem + \emph{},(<cit.> )\nLet G be a graph with |L(G)|=1 and ...,(<cit.>,B_MATH_TERM )\nLet G be a graph with |L(G)|=1 ...,B_MATH_TERM O O O O O O O O O O O O O O O O O ...
893,"\cite[Sec.\thinspace 4]{Gropp-nk;0,1-matrix",\n\label{Th1_2L+1}\emph{\cite[Sec.\thinspace 4...,/home/shufan/que_tex/1203.0709.tex,theorem + \emph{},\n<cit.>\n\n\n\n (i) Any Golomb ruler G_k of ...,<cit.>,\nB_MATH_TERM I_MATH_TERM\n\n\n\n (i) Any Gol...,B_MATH_TERM I_MATH_TERM O O O O O O O O O O O ...
2287,"\textbf{(Guth, Katz, \cite[Theorem 4.1]{Guth_K...","\emph{\textbf{(Guth, Katz, \cite[Theorem 4.1]{...",/home/shufan/que_tex/1203.3735.tex,theorem + \emph{},"(Guth, Katz, <cit.> ) Let 𝔊 be a finite set of...","(Guth, Katz, <cit.>",B_MATH_TERM I_MATH_TERM I_MATH_TERM I_MATH_TER...,B_MATH_TERM I_MATH_TERM I_MATH_TERM I_MATH_TER...
2288,"\textbf{(Guth, Katz, \cite[Corollary 2.5]{Guth...","\emph{\textbf{(Guth, Katz, \cite[Corollary 2.5...",/home/shufan/que_tex/1203.3735.tex,theorem + \emph{},"(Guth, Katz, <cit.> ) (Corollary of Bézout's T...","(Guth, Katz, <cit.>",B_MATH_TERM I_MATH_TERM I_MATH_TERM I_MATH_TER...,B_MATH_TERM I_MATH_TERM I_MATH_TERM I_MATH_TER...
2498,\cite[Theorem 1]{eseq06,\emph{\cite[Theorem 1]{eseq06}}\label{charact...,/home/shufan/que_tex/lagrange.tex,theorem + \emph{},<cit.>\n A permutation π on [n] is 35241-av...,<cit.>,B_MATH_TERM I_MATH_TERM\n A permutation π o...,B_MATH_TERM I_MATH_TERM O O O O O O O O O O O ...
2908,(\cite[Theorem 1.1]{ca,\emph{(\cite[Theorem 1.1]{ca} ).}\n\label{thca...,/home/shufan/que_tex/new.tex,theorem + \emph{},(<cit.> ).\n The poset (S_n) is homotopy Cohen...,(<cit.>,B_MATH_TERM I_MATH_TERM ).\n The poset (S_n) i...,B_MATH_TERM I_MATH_TERM O O O O O O O O O O O ...
2909,(\cite[Theorem 1.2]{ca,\emph{(\cite[Theorem 1.2]{ca} ).}\n\label{thca...,/home/shufan/que_tex/new.tex,theorem + \emph{},(<cit.> ).\n\nLet P̅_n=(S_n){e}. The reduced E...,(<cit.>,B_MATH_TERM I_MATH_TERM ).\n\nLet P̅_n=(S_n){e...,B_MATH_TERM I_MATH_TERM O O O O O O O O O O O ...
3689,\cite[Corollary 9.7]{Q,\emph{\cite[Corollary 9.7]{Q}}\n\label{Quille...,/home/shufan/que_tex/new_full.tex,theorem + \emph{},<cit.>\n\n Let P and Q be graded posets. Let ...,<cit.>,B_MATH_TERM I_MATH_TERM\n\n Let P and Q be gr...,B_MATH_TERM I_MATH_TERM O O O O O O O O O O O ...


In [70]:
df_clean = df_clean[~df_clean['plain_text_term'].str.contains('<cit')]

In [71]:
df_clean[df_clean['plain_text_term'].str.contains('<ref')]

,term,def,file_name,extraction_rule,plain_text_def,plain_text_term,masked_plain_text_def,labeled_def
316,Statement of Theorem \ref{thm:singleton_b_is_a...,\n%\label{thm:singleton_b_is_a_prime}\n\emph{S...,/home/shufan/que_tex/1803.03954.tex,theorem + \emph{},\n\nStatement of Theorem <ref>: Let n be a pos...,Statement of Theorem <ref>,\n\nB_MATH_TERM I_MATH_TERM I_MATH_TERM I_MATH...,B_MATH_TERM I_MATH_TERM I_MATH_TERM O O O O O ...
894,(\ref{eq3.2_Ah_Bh;\ref{def3.2_Asigma;circulant...,"\n\label{th3.2_Asigma-s}Let $(a_{1},a_{2},\ldo...",/home/shufan/que_tex/1203.0709.tex,theorem + \emph{},"\nLet (a_1,a_2,… ,a_k) be a\n(v,k) modular Gol...",(<ref>,"\nLet (a_1,a_2,… ,a_k) be a\n(v,k) modular Gol...",O O O O O O O O O O O O O O O O O O O O O O O ...
908,(\ref{LB_M1_general,\label{LB_M1_general_theorem}\nFor $n \ge 2$\n...,/home/shufan/que_tex/Goubko.tex,theorem + \emph{},\nFor n ≥ 2\n\n C(T)≥C(n):=nc(1)+(n-2)c(Δ(n...,(<ref>,\nFor n ≥ 2\n\n C(T)≥C(n):=nc(1)+(n-2)c(Δ(n...,O O O O O O O O O O O O O O O O O O O O O O O ...
1573,(;);\ref{Fig.1;(i);(ii);(;);(iv);(v),\n\label{CLTORDP}There exist five distinct tor...,/home/shufan/que_tex/One_Sing.tex,theorem + \emph{},\nThere exist five distinct toric del Pezzo su...,(;);<ref>,\nThere exist five distinct toric del Pezzo su...,O O O O O O O O O O O O O O O O O O O O O O O ...
1574,(;);:\smallskip;(i);(ii) ;conv;conv;conv;Figur...,\n\label{MAIN1}Let $X_{Q}$ be a toric log del ...,/home/shufan/que_tex/One_Sing.tex,theorem + \emph{},\nLet X_Q be a toric log del Pezzo surface (as...,(;);:;(i);(ii) ;conv;conv;conv;Figure <ref>,\nLet X_Q be a toric log del Pezzo surface B_M...,O O O O O O O O O O O O O O O O O O O O O O O ...
1644,(\ref{one,\label{sun-complexity}\nDetermining whether th...,/home/shufan/que_tex/1801.09170.tex,theorem + \emph{},\nDetermining whether the multiplicity (<ref> ...,(<ref>,\nDetermining whether the multiplicity B_MATH_...,O O O O B_MATH_TERM O O O O O O O O O O O O O ...
1647,(\ref{two;(\ref{three,\n Determining whether multiplicities \emph{(\...,/home/shufan/que_tex/1801.09170.tex,theorem + \emph{},\n Determining whether multiplicities (<ref> )...,(<ref>,\n Determining whether multiplicities B_MATH_T...,O O O B_MATH_TERM O O B_MATH_TERM O O O O O O ...
2092,See \autoref{prelims1,\label{mainthm_morally}\label{undom_td}\nEvery...,/home/shufan/que_tex/undom_td150218.tex,theorem + \emph{},"\nEvery graph has a tree-decomposition (T,) of...",See <ref>,"\nEvery graph has a tree-decomposition (T,) of...",O O O O O O O O O O O O O O O O O O O O O O B_...
2998,(\ref{eq:faceboundaries,\nLet $\pi$ be a rotation system of a graph $...,/home/shufan/que_tex/hex-braces_arxiv.tex,theorem + \emph{},\nLet π be a rotation system of a graph G. Th...,(<ref>,\nLet π be a rotation system of a graph G. Th...,O O O O O O O O O O O O O O O O O O O O O O O ...
3142,(\ref{eq:psi-def;(\ref{eq:psi-def,\nIf $0<b<1$ and $a>0$ then the function $\mu\...,/home/shufan/que_tex/1509.05120.tex,theorem + \emph{},\nIf 0<b<1 and a>0 then the function μ→x^-a-μ(...,(<ref>,\nIf 0<b<1 and a>0 then the function μ→x^-a-μ(...,O O O O O O O O O O O O O O O O O O O O O O O ...


In [72]:
df_clean = df_clean[~df_clean['plain_text_term'].str.contains('<ref')]

In [73]:
df_clean.to_csv('cleaned_plain-text_term+combined_no_ref_no_cit_def_labeled.csv',index=False)

## TODO: why some label strings have no B-MATH_TERM? And why some of these contains I-MATH_TERM???

In [30]:
#df = pd.read_csv("cleaned_plain-text_labeled_term+combined_no_ref_no_cit_def_same_len_only.csv", delimiter=',')
df = pd.read_csv("cleaned_plain-text_labeled_term+combined_no_ref_no_cit_def_same_len_only_must_conatin_B.csv", delimiter=',')
len(df)

13692

In [26]:
O_then_I = df[df['labeled_def'].str.contains('O I_MATH_TERM')]

In [29]:
must_b = df[df['labeled_def'].str.contains('B_MATH_TERM')]
must_b = must_b[~must_b['labeled_def'].str.contains('O I_MATH_TERM')]
must_b.to_csv("cleaned_plain-text_labeled_term+combined_no_ref_no_cit_def_same_len_only_must_conatin_B.csv", index=False)

In [28]:
O_then_I.count()

term                     48
def                      48
file_name                48
extraction_rule          48
plain_text_def           48
plain_text_term          48
masked_plain_text_def    48
labeled_def              48
dtype: int64

In [ ]:
no_b = df[~df['labeled_def'].str.contains('B_MATH_TERM')]
no_b 

In [ ]:
no_b.to_csv("cleaned_plain-text_labeled_term+combined_no_ref_no_cit_def_same_len_only_why_no_B.csv", index=False)

no_b_but_i = no_b[no_b['labeled_def'].str.contains('I_MATH_TERM')]
no_b_but_i

In [19]:
no_b_but_i.to_csv("cleaned_plain-text_labeled_term+combined_no_ref_no_cit_def_same_len_only_why_no_B_but_I.csv",index= False)

## TODO: XXX-B_MATH_TERM are lost 
def 19	A \emph{$Z$-oriented spanning forest} of $G$ is an oriented subgraph $F$ of $G$ such that...

This line is extracted with the definition[] rule, as in the [] it is noted as "Oriented spanning forest", in the {},it is written "\emph{$Z$-oriented spanning forest}"
so the masked plained text is:
	
A Z-B_MATH_TERM I_MATH_TERM I_MATH_TERM of G is an oriented subgraph F of G such that...

However, as we have dealed with \emph{} apart, this text shouble be doubled

## TODO: Notions \textit{E} and \textit{F}
Let $\mathcal{C}$ be a conic inscribed in an octagon $A B C D E F
G H$, let $\textit{E}$ be a conic touching some four sides of the
octagon, and $\textit{F}$ be a conic touching the remaining four
sides of octagons. 

## TODO:   of \emph{Type} $\cal M$
\label{definition3}(Auxiliary definition) Let $q+1$ be an odd integer. An edge-coloring $\varsigma\colon E(K_{q+2}) \rightarrow [q+1]$ is of \emph{Type} $\cal M$, if for every $i\in [q+1]$ the set of edges $\varsigma^{-1}(i)$ is a perfect matching of $K_{q+2}$.

## TODO: These are not terms
	plain_text_term
76	atoms;supp(p)-{p_1}∈𝒮_P;supp

	masked_plain_text_def
76	
Let P be a finite atomic lattice. Then P is super-atomic if and only if for any p∈ P-B_MATH_TERM(P)⋃{0}, there exist two atoms p_1,  p_2 in P such that p= p_1∨ p_2 satisfying that B_MATH_TERM andB_MATH_TERM(p)-{p_2}∈𝒮_P.


## TODO: The "at" is not a term, the terms should be "simple local complement of S at i"
\label{lc} Let $S$ be a symmetric $n\times n$ matrix, and suppose $1\leq i\leq
n$. Then the \emph{simple local complement of }$S$ \emph{at} $i$ is the
symmetric $GF(2)$-matrix $S^{i}$ obtained from $S$ as follows: whenever $i\neq
j\neq k\neq i$ and $s_{ij}\neq0\neq s_{ik}$, toggle $s_{jk}$.

## TODO: \textit{n} might be $n$

## TODO: \emph{f}ar-\emph{a}part \emph{n}ine
A subset $\mathcal{F}$ of nine points of a $\mathcal{S}_{3}(2)$ is a
\emph{fan} (= \emph{f}ar-\emph{a}part \emph{n}ine) if no two points of
$\mathcal{F}$ lie on the same generator. (So if $\mathcal{F}$ is a fan for a
$\mathcal{S}_{3}(2)$ then the 3 generators through each of the 9 points of
$\mathcal{F}$ account for all $3\times9=27$ generators of $\mathcal{S}%
_{3}(2).$)

In [ ]:
combi_pattern = r'(\\emph\{.*?\}(a-zA-Z|$))|(\\textit\{.*?\}(a-zA-Z|$))'

df_clean[df_clean['def'].str.contains(combi_pattern)]#.to_csv('test_more_combi.csv')

In [ ]:
df_clean[df_clean['def'].str.contains(combi_pattern)].to_csv('test_more_combi.csv',index=False)

# Check plain text length & labeled_def length !!!

In [74]:
df_clean = pd.read_csv('cleaned_plain-text_term+combined_no_ref_no_cit_def_labeled.csv')
df_clean.tail()

,term,def,file_name,extraction_rule,plain_text_def,plain_text_term,masked_plain_text_def,labeled_def
14914,preferred direction;special claw corresponding...,\label{definition:SpecialClaw} Consider $\lang...,/home/shufan/que_tex/CosetsVoltagesAndDerivedE...,definition + \textit{},"Consider ⟨ G→ S, α→ A⟩, two adjacent vertices...",preferred direction;special claw corresponding...,"Consider ⟨ G→ S, α→ A⟩, two adjacent vertices...",O O O O O O O O O O O O O O O O O O O O O O O ...
14915,Cyclicity classes,\label{d:cyclasses}\nAccording to Theorem~\ref...,/home/shufan/que_tex/1307.3716.tex,definition[],"\nAccording to Theorem <ref>, ^d has exactly d...",Cyclicity classes,"\nAccording to Theorem <ref>, ^d has exactly d...",O O O O O O O O O O O O O O O O O O O O O O O ...
14916,Wielandt number,By {\em Wielandt number} we mean the following...,/home/shufan/que_tex/1307.3716.tex,definition[],By Wielandt number we mean the following funct...,Wielandt number,By B_MATH_TERM I_MATH_TERM we mean the followi...,O B_MATH_TERM I_MATH_TERM O O O O O O O O O O O O
14917,Girth,"\label{d:girth}\nThe {\em girth\/} of $\digr$,...",/home/shufan/que_tex/1307.3716.tex,definition[],"\nThe girth of , denoted it by g(),\nis the sm...",Girth,"\nThe B_MATH_TERM of , denoted it by g(),\nis ...",O B_MATH_TERM O O O O O O O O O O O O O O O O
14918,Critical graph,\label{d:crit}\nLet $A\in\Rpnn$. The cycles of...,/home/shufan/que_tex/1307.3716.tex,definition[],\nLet A∈. The cycles of (A) at which the maxim...,Critical graph,\nLet A∈. The cycles of (A) at which the maxim...,O O O O O O O O O O O O O O O O O O O O O O O ...


In [75]:
df_clean['diff_tokens'] = df_clean.apply(lambda row: len(row['plain_text_def'].split()) != len(row['labeled_def'].split()), axis=1)
df_clean[df_clean['diff_tokens']].count()

term                     774
def                      774
file_name                774
extraction_rule          774
plain_text_def           774
plain_text_term          774
masked_plain_text_def    774
labeled_def              774
diff_tokens              774
dtype: int64

In [76]:
df_clean[df_clean['diff_tokens']].to_csv("diff_tokens.csv")

In [9]:
len("B_MATH_TERM")

11

In [77]:
MATH_TERM_phrases = {"I_MATH_TERM,"}

for sent in df_clean['masked_plain_text_def']:
    tokens = sent.split()
    for token in tokens:
        if ('B_MATH_TERM' in token or 'I_MATH_TERM' in token) and len(token) > 11:
            MATH_TERM_phrases.add(token)

In [78]:
#note this phrases to a txt file
my_set_str = str(MATH_TERM_phrases)

# Open a new file for writing
with open('tokens_that_contains_B_I_MATH_TERM.txt', 'w') as f:
    # Write the string to the file
    f.write(my_set_str)

In [79]:
df_clean['diff_tokens'] = df_clean.apply(lambda row: len(row['masked_plain_text_def'].split()) != len(row['labeled_def'].split()), axis=1)
df_clean[df_clean['diff_tokens']].count()

term                     0
def                      0
file_name                0
extraction_rule          0
plain_text_def           0
plain_text_term          0
masked_plain_text_def    0
labeled_def              0
diff_tokens              0
dtype: int64

In [81]:
df_clean['diff_tokens'] = df_clean.apply(lambda row: len(row['plain_text_def'].split()) != len(row['labeled_def'].split()), axis=1)
df_clean[df_clean['diff_tokens']].count()

term                     774
def                      774
file_name                774
extraction_rule          774
plain_text_def           774
plain_text_term          774
masked_plain_text_def    774
labeled_def              774
diff_tokens              774
dtype: int64

In [85]:
df_clean = df_clean[~df_clean['diff_tokens']]
df_clean = df_clean.drop(columns={'diff_tokens'})
df_clean

,term,def,file_name,extraction_rule,plain_text_def,plain_text_term,masked_plain_text_def,labeled_def
0,admissible $G$-cover,\label{def_admissibleGcover}\nLet $G$ be a fin...,/home/shufan/que_tex/abelian_tropical_covers.tex,definition + \emph{},\nLet G be a finite group and let X→ S be a fa...,admissible G-cover,\nLet G be a finite group and let X→ S be a fa...,O O O O O O O O O O O O O O O O O O O O O O O ...
1,light reduction pair;heavy reduction pair,"\n Let $\pi\in\fs_n$ and let $x,y$ be a pair ...",/home/shufan/que_tex/Post071005.tex,definition + \emph{},"\n Let π∈_n and let x,y be a pair of rooks th...",light reduction pair;heavy reduction pair,"\n Let π∈_n and let x,y be a pair of rooks th...",O O O O O O O O O O O O O O O O O O O O O O B_...
2,Local modification rule,\label{lacma} A \emph{local modification rule...,/home/shufan/que_tex/hereditary_submit.tex,definition[],"A local modification rule is a pair (A,T), w...",Local modification rule,A B_MATH_TERM I_MATH_TERM I_MATH_TERM is a p...,O B_MATH_TERM I_MATH_TERM I_MATH_TERM O O O O ...
3,Vertex sets,\label{vertset} A \emph{vertex set} is any se...,/home/shufan/que_tex/hereditary_submit.tex,definition[],A vertex set is any set which is at most cou...,Vertex sets,A vertex set is any set which is at most cou...,O O O O O O O O O O O O O O O O B_MATH_TERM O ...
4,Palettes,A \emph{finite palette} is a sequence $K = (K_...,/home/shufan/que_tex/hereditary_submit.tex,definition[],A finite palette is a sequence K = (K_j)_j=0^∞...,Palettes,A finite palette is a sequence K = (K_j)_j=0^∞...,O O O O O O O O O O O O O O O O O O O O O O O ...
...,...,...,...,...,...,...,...,...
14914,preferred direction;special claw corresponding...,\label{definition:SpecialClaw} Consider $\lang...,/home/shufan/que_tex/CosetsVoltagesAndDerivedE...,definition + \textit{},"Consider ⟨ G→ S, α→ A⟩, two adjacent vertices...",preferred direction;special claw corresponding...,"Consider ⟨ G→ S, α→ A⟩, two adjacent vertices...",O O O O O O O O O O O O O O O O O O O O O O O ...
14915,Cyclicity classes,\label{d:cyclasses}\nAccording to Theorem~\ref...,/home/shufan/que_tex/1307.3716.tex,definition[],"\nAccording to Theorem <ref>, ^d has exactly d...",Cyclicity classes,"\nAccording to Theorem <ref>, ^d has exactly d...",O O O O O O O O O O O O O O O O O O O O O O O ...
14916,Wielandt number,By {\em Wielandt number} we mean the following...,/home/shufan/que_tex/1307.3716.tex,definition[],By Wielandt number we mean the following funct...,Wielandt number,By B_MATH_TERM I_MATH_TERM we mean the followi...,O B_MATH_TERM I_MATH_TERM O O O O O O O O O O O O
14917,Girth,"\label{d:girth}\nThe {\em girth\/} of $\digr$,...",/home/shufan/que_tex/1307.3716.tex,definition[],"\nThe girth of , denoted it by g(),\nis the sm...",Girth,"\nThe B_MATH_TERM of , denoted it by g(),\nis ...",O B_MATH_TERM O O O O O O O O O O O O O O O O


In [86]:
df_clean.to_csv("cleaned_plain-text_labeled_term+combined_no_ref_no_cit_def_same_len_only.csv", index=False)

In [87]:
df_clean['extraction_rule'].value_counts()

definition + \emph{}      10360
definition + \textit{}     1775
definition[]               1271
theorem + \emph{}           629
theorem + \textit{}         110
Name: extraction_rule, dtype: int64

In [90]:
df_clean['diff_tokens'] = df_clean.apply(lambda row: len(row['plain_text_def'].split()) != len(row['labeled_def'].split()), axis=1)
df_clean[df_clean['diff_tokens']].count()

term                     0
def                      0
file_name                0
extraction_rule          0
plain_text_def           0
plain_text_term          0
masked_plain_text_def    0
labeled_def              0
diff_tokens              0
dtype: int64